## Finalizing Bldg Permit Data for Creating Visualizations
- Discovered when starting to create visualizations that I needed one cleaned dataset with **both** types of permits Applications and Permits; and I only needed *some* of the cleaning steps used in the prior notebooks, and in a different sequence than they had been applied. I may need those steps later, so I'm leaving those notebooks intact.
- SOLUTION: Creating new notebook 3_finalize_data and will only do the steps needed to get final, single dataset.
- NAMING: To differentiatl these two bldg. permit dfs from prior notebooks, using these names:
    - df1_applic (permit applications)  
    - df2_issued (permits issued)  
    - df3_all (final, merged file)  
- PREPARE
    1. Column name cleanup
    2. Data types cleanup
    3. New column: 'category' (Residential, Commercial, Other)
    4. New column: 'type_subtype' concatenation
    5. Drop addresses with "0" house number
    6. Pull lat/lon from mapped_location
    7. Drop rows with null lat/lon
    8. Sort by address (prerequsite to dropping duplicates)
    9. Drop duplicates
    10. Convert to geodataframe
    11. Keep only addresses within Davidson County
    12. Add Neighborhood Association names
    13. Merge
    14. Export

In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd   # Prerequisite: Anotes_and_docsctivate geospatial environment via Conda Prompt
import matplotlib.pyplot as plt  
import folium                   
from folium.plugins import MarkerCluster
import datetime
from shapely.geometry import Point  
import re # To use RegEx to pull ot lat/long from building permit applications/issued

## Reading in raw data for building permits

In [6]:
# Building Dept. Permit Applications

df1_applic = pd.read_csv('../data/raw/Building_Permit_Applications_2020_06_05.csv')
df1_applic.head(2)

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Construction Cost,Address,City,State,ZIP,Subdivision / Lot,Contact,Permit Type,Permit Subtype,IVR Tracking #,Purpose,Council District,Mapped Location
0,T2020016213,Building Residential - New,Single Family Residence,10216006100,03/11/2020,NaN,NaN,748 DARDEN PL,NASHVILLE,TN,37205,LOT 168 SEC 9 PT 2 HILLWOOD EST,Kingdom Builders of Tennesse,CARN,CAA01R301,3781725,New Single family dwelling. REJECTED: APPLICA...,23.0,"748 DARDEN PL\nNASHVILLE, TN 37205\n(36.125944..."
1,T2019073204,Building Moving Permit,Moving Permit - Residential,04600002700,12/02/2019,NaN,2500.0,4836 BULL RUN RD,ASHLAND CITY,TN,37015,N OF BULL RUN RD W OF OLD HICKORY BLVD,CLAYTON HOMES #054,CAMV,CAZ09A001,3736813,Move existing mobile home from property out of...,1.0,"4836 BULL RUN RD\nASHLAND CITY, TN 37015\n(36...."


In [8]:
df1_applic.shape

(3106, 19)

In [15]:
# Building Dept. Permits Issued
# low_memory = False was added to remove a low-memory warning. Doing this prevents the
# system from trying to assign dtypes until after the full file has been read
# Resource: https://tinyurl.com/stackoverflow-low-memory

df2_issued = pd.read_csv('../data/raw/Building_Permits_Issued_2020_06_05.csv'
                        , low_memory=False
                        )
df2_issued.head(2)

,Permit #,Permit Type Description,Permit Subtype Description,Parcel,Date Entered,Date Issued,Construction Cost,Address,City,State,ZIP,Subdivision / Lot,Contact,Permit Type,Permit Subtype,IVR Tracking #,Purpose,Council District,Census Tract,Mapped Location
0,2019070460,Building Residential - New,Single Family Residence,058100C04900CO,11/18/2019,12/09/2019,270585.0,1037 LAWSONS RIDGE DR,NASHVILLE,TN,37218,LOT 49 CARRINGTON PLACE PH 5,CELEBRATION HOMES LLC,CARN,CAA01R301,3733056,To construct a single family residence of 2402...,1.0,37010105.0,"1037 LAWSONS RIDGE DR\nNASHVILLE, TN 37218"
1,2020016259,Building Residential - Rehab,Single Family Residence,160150A07000CO,03/12/2020,03/12/2020,12000.0,210 HEARTHSTONE MANOR LN,BRENTWOOD,TN,37027,UNIT 70 HEARTHSTONE MANOR CONDOMINIUM PHASE 4,ACCESS & MOBILITY INC,CARR,CAA01R301,3781961,to install a new elevator/platform lift from g...,4.0,37018803.0,"210 HEARTHSTONE MANOR LN\nBRENTWOOD, TN 37027\..."


In [16]:
df2_issued.shape

(33909, 20)

## 1. Column name cleanup

## 2. Data types cleanup

## 3. New column: 'category' (Residential, Commercial, Other)

## 4. New column: 'type_subtype' concatenation

## 5. Drop addresses with "0" house number

## 6. Pull lat/lon from mapped_location

## 7. Drop rows with null lat/lon

## 8. Sort by address (prerequsite to dropping duplicates)

## 9. Drop duplicates

## 10. Convert to geodataframe

## 11. Keep only addresses within Davidson County

## 12. Add Neighborhood Association names

## 13. Merge

## 14. Export

### *To change row truncation, as needed*

In [17]:
# To get full results that aren't truncated, set to high number (default is 50):
pd.options.display.max_rows = 50
print(pd.options.display.max_rows)

50
